In [21]:
# -*- coding: utf-8 -*-
from __future__ import division, absolute_import

import tldextract
#import tflearn
import os
import pickle
import numpy as np
import keras
#from tflearn.data_utils import to_categorical, pad_sequences
#import tensorflow as tf
#import tflearn
#from tflearn.layers.core import input_data, dropout, fully_connected
#from tflearn.layers.conv import conv_1d, global_max_pool
#from tflearn.layers.merge_ops import merge
#from tflearn.layers.estimator import regression
#from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split
import pickle
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Embedding
from keras.layers import LSTM,Bidirectional,GRU
SRC_IP_IDX = 3-1               
DST_IP_IDX = 4-1               
SRC_PORT_IDX = 5-1             
DST_PORT_IDX = 6-1             
PROTOCOL_IDX = 7-1             
DNS_QUERY_NAME_IDX = 55-1 # domain
DNS_REQUEST_TYPE = 56-1
DNS_DOMAIN_TTL = 59-1
DNS_REPLY_IPV4IP = 60-1        
DNS_REPLY_IPV6IP = 61-1        
DNS_REPLY_RRTYPE = 62-1        
DNS_REQUEST_LEN  = 88-1        
DNS_REPLY_LENGTH = 90-1        
  

def iterbrowse(path):
    for home, dirs, files in os.walk(path):
        for filename in files:
            yield os.path.join(home, filename)

def pad_sequences(X, maxlen, value=0):
    S=[]
    for x in X:
        xlen = len(x)
        if xlen < maxlen:
            x.extend([value]*(maxlen-xlen))
        else:
            x = x[:maxlen]
        S.append(x)
    return S
def extract_domain(domain):
    suffix = {'.com','.la','.io', '.co', '.cn','.info', '.net', '.org','.me', '.mobi', '.us', '.biz', '.xxx', '.ca', '.co.jp', '.com.cn', '.net.cn', '.org.cn', '.mx','.tv', '.ws', '.ag', '.com.ag', '.net.ag', '.org.ag','.am','.asia', '.at', '.be', '.com.br', '.net.br', '.name', '.live', '.news', '.bz', '.tech', '.pub', '.wang', '.space', '.top', '.xin', '.social', '.date', '.site', '.red', '.studio', '.link', '.online', '.help', '.kr', '.club', '.com.bz', '.net.bz', '.cc', '.band', '.market', '.com.co', '.net.co', '.nom.co', '.lawyer', '.de', '.es', '.com.es', '.nom.es', '.org.es', '.eu', '.wiki', '.design', '.software', '.fm', '.fr', '.gs', '.in', '.co.in', '.firm.in', '.gen.in', '.ind.in', '.net.in', '.org.in', '.it', '.jobs', '.jp', '.ms', '.com.mx', '.nl','.nu','.co.nz','.net.nz', '.org.nz', '.se', '.tc', '.tk', '.tw', '.com.tw', '.idv.tw', '.org.tw', '.hk', '.co.uk', '.me.uk', '.org.uk', '.vg'}

    domain = domain.lower()
    names = domain.split(".")
    if len(names) >= 3:
        if ("."+".".join(names[-2:])) in suffix:
            return ".".join(names[-3:]), ".".join(names[:-3])
        elif ("."+names[-1]) in suffix:
            return ".".join(names[-2:]), ".".join(names[:-2])
    #print ("New domain suffix found. Use tld extract domain...")

    pos = domain.rfind("/")
    if pos >= 0: # maybe subdomain contains /, for dns tunnel tool
        ext = tldextract.extract(domain[pos+1:])
        subdomain = domain[:pos+1] + ext.subdomain
    else:
        ext = tldextract.extract(domain)
        subdomain = ext.subdomain
    if ext.suffix:
        mdomain = ext.domain + "." + ext.suffix
    else:
        mdomain = ext.domain
    return mdomain, subdomain


def filter_metadata_dns(data):
    if(len(data) < 91):
        return False

    protol  = data[PROTOCOL_IDX]
    dstport = data[DST_PORT_IDX]
    dstip   = data[DST_IP_IDX]
    qname   = data[DNS_QUERY_NAME_IDX]

    if '' == qname or '' == dstip:
        return False
    if '17' == protol and ('53' == dstport):
        return True
    return False


def metadata2_domain_data(log): 
    data = log.split('^')
    if not filter_metadata_dns(data):
        return None, None
    domain = data[DNS_QUERY_NAME_IDX]
    mdomain, subdomain = extract_domain(domain)
    return (mdomain, subdomain)


def get_local_data(tag="labeled"):
    data_path = "./sample_data"
    black_data, cdn_data, white_data = [], [], []    
    for dir_name in ("black", "cdn", "white"):
        dir_path = "%s/%s_%s" % (data_path, tag, dir_name)

        for path in iterbrowse(dir_path):
            print(path)
            with open(path) as f:
                for line in f:
                    mdomain, subdomain = metadata2_domain_data(line)
                    if subdomain is not None:
                        if "white" in path:
                            white_data.append(subdomain)
                        elif "cdn" in path:
                            cdn_data.append(subdomain)
                        elif "black" in path and "pcap" in path:
                            black_data.append(subdomain)
                        else:
                            pass
                            #print ("pass path:", path)
                    #else:
                    #    print ("unknown line:", line, " in file:", path)
    return black_data, cdn_data, white_data


class LABEL(object):
    white = 0
    cdn = 1
    black = 2


def get_data():
    black_x, cdn_x, white_x = get_local_data()
    black_y, cdn_y, white_y = [LABEL.black]*len(black_x), [LABEL.cdn]*len(cdn_x), [LABEL.white]*len(white_x)

    X = black_x + cdn_x + white_x
    labels = black_y + cdn_y + white_y

    # Generate a dictionary of valid characters
    valid_chars = {x:idx+1 for idx, x in enumerate(set(''.join(X)))}

    max_features = len(valid_chars) + 1
    #print "max_features:", max_features
    maxlen = np.max([len(x) for x in X])
    #print "max_len:", maxlen
    maxlen = min(maxlen, 256)

    # Convert characters to int and pad
    X = [[valid_chars[y] for y in x] for x in X]
    X = pad_sequences(X, maxlen=maxlen, value=0.)

    # Convert labels to 0-1
    #print(labels)
    Y = keras.utils.to_categorical(labels, num_classes=3)
    
    volcab_file = "volcab.pkl"
    output = open(volcab_file, 'wb')
    # Pickle dictionary using protocol 0.
    data = {"valid_chars": valid_chars, "max_len": maxlen, "volcab_size": max_features}
    pickle.dump(data, output)
    output.close()

    return X, Y, maxlen, max_features
def build_model_BiRNN(max_len, volcab_size):
    """Build Bi-RNN model"""
    model = Sequential()
    model.add(Embedding(input_dim=volcab_size,
                        output_dim=64,
                        input_length=max_len))
    model.add(Bidirectional(GRU(16)))          
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    #model.summary()
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

def get_rnn_model(max_len, volcab_size):
    # Network building
    net = tflearn.input_data([None, max_len])
    net = tflearn.embedding(net, input_dim=volcab_size, output_dim=64)
    net = tflearn.lstm(net, 64, dropout=0.8)
    net = tflearn.fully_connected(net, 3, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
			     loss='categorical_crossentropy')
    model = tflearn.DNN(net, tensorboard_verbose=0)
    return model


def get_cnn_model(max_len, volcab_size):
    # Building convolutional network
    network = tflearn.input_data(shape=[None, max_len], name='input')
    network = tflearn.embedding(network, input_dim=volcab_size, output_dim=64)
    branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool(network)
    network = dropout(network, 0.5)
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    model = tflearn.DNN(network, tensorboard_verbose=0)
    return model


def run():
    X, Y, max_len, volcab_size = get_data()

    print("X len:", len(X), "Y len:", len(Y))
    trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=42)
    print(len(trainX))
    trainX=np.array(trainX)
    testX=np.array(testX)
    trainY=np.array(trainY)
    testY=np.array(testY)
    #print(trainX[:1])
    #print(trainY[:1])
    #print(testX[-1:])
    #print(testY[-1:])
    #print(type(trainX))
    model = build_model_BiRNN(max_len, volcab_size)
    model.summary()
    model.fit(trainX, trainY, validation_data=(testX, testY), verbose=1,batch_size=32,epochs=1)
    #model.fit(trainX, trainY,batch_size=32,epochs=2,validation_data=(testX, testY))
    filename = 'dnstunnel.module'
    model.save(filename)

    model=load_model(filename)
    #print "Just review 3 sample data test result:"
    result = model.predict(testX[0:3])
    print(result)


if __name__ == "__main__":
    run()


./sample_data/labeled_black/dns2tcp_cmd.pcap.txt
./sample_data/labeled_black/dns2tcp_sendfile9.pcap.txt
./sample_data/labeled_black/dns2tcp_when_use_ssh.txt
./sample_data/labeled_black/dnscapy_scp.pcap.txt
./sample_data/labeled_black/dnscapy_scp2.pcap.txt
./sample_data/labeled_black/dnscat2_when_exec_command_rm_file.txt
./sample_data/labeled_black/dnscat2_when_idle.txt
./sample_data/labeled_black/download_dnscat2_file12.pcap.txt
./sample_data/labeled_black/download_dnscat2_file13.pcap.txt
./sample_data/labeled_black/download_dnscat2_file14.pcap.txt
./sample_data/labeled_black/iodine_direct_scp17_base128.pcap.txt
./sample_data/labeled_black/iodine_direct_scp3_base64.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh4_base32.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh6_base128.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh6_base64.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh6_base64u.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh9_base32_again.pca

In [5]:
X, Y, max_len, volcab_size = get_data()


./sample_data/labeled_black/dns2tcp_cmd.pcap.txt
./sample_data/labeled_black/dns2tcp_sendfile9.pcap.txt
./sample_data/labeled_black/dns2tcp_when_use_ssh.txt
./sample_data/labeled_black/dnscapy_scp.pcap.txt
./sample_data/labeled_black/dnscapy_scp2.pcap.txt
./sample_data/labeled_black/dnscat2_when_exec_command_rm_file.txt
./sample_data/labeled_black/dnscat2_when_idle.txt
./sample_data/labeled_black/download_dnscat2_file12.pcap.txt
./sample_data/labeled_black/download_dnscat2_file13.pcap.txt
./sample_data/labeled_black/download_dnscat2_file14.pcap.txt
./sample_data/labeled_black/iodine_direct_scp17_base128.pcap.txt
./sample_data/labeled_black/iodine_direct_scp3_base64.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh4_base32.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh6_base128.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh6_base64.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh6_base64u.pcap.txt
./sample_data/labeled_black/iodine_direct_ssh9_base32_again.pca

In [6]:
print(np.shape(X))
print(np.shape(Y))

(4439, 256)
(4439, 3)


In [7]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=42)
#print(type(trainX[0][0]))
#print(len(trainX))
#print(len(trainY))
#print(type(trainY))

In [8]:
print(np.shape(trainX))
print(np.shape(trainY))

(3551, 256)
(3551, 3)


In [9]:
trainX=np.array(trainX)
#print((trainX[0:2]))
#print(type(trainX))
#print(type(trainX[0:2]))
testX=np.array(testX)
print(np.shape(trainX))
trainY=np.array(trainY)
#trainY=trainY.reshape(trainY.shape[0],1)
#testY=np.array(testY)
#testY=testY.reshape(testY.shape[0],1)
#print((trainY))
print(np.shape(trainY))
#trainY=np.mat(trainY).flatten().T
#testY=np.mat(testY).flatten().T
#print(np.shape(trainY))
#print(trainY)

(3551, 256)
(3551, 3)


In [10]:
print(type(trainX))
print(type(trainY))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [11]:
from keras.layers import LSTM,Bidirectional,GRU
def build_model_BRNN(max_len, volcab_size):
    """Build Bi-RNN model"""
    model = Sequential()
    model.add(Embedding(input_dim=volcab_size,
                        output_dim=64,
                        input_length=max_len))
    model.add(Bidirectional(GRU(16)))          
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    #model.summary()
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model
model = build_model_BRNN(max_len, volcab_size)

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 256, 64)           2752      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32)                7776      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 10,627
Trainable params: 10,627
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(trainX, trainY,batch_size=32,epochs=2,validation_data=(testX, testY))
#model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, batch_size=32)

Train on 3551 samples, validate on 888 samples
Epoch 1/2
3551/3551 [==============================] - 60s 17ms/step - loss: 0.7192 - acc: 0.7398 - val_loss: 0.3374 - val_acc: 0.8986
Epoch 2/2
3551/3551 [==============================] - 56s 16ms/step - loss: 0.2231 - acc: 0.9358 - val_loss: 0.1479 - val_acc: 0.9595


In [33]:
print(testX[0:3])
result = model.predict(testX[0:3])
print(result)

[[ 7. 41. 23. 35.  3. 38.  6. 19. 18. 19. 42.  8.  1. 11. 38.  8. 19. 30.
  38.  6. 33.  5. 18. 17.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0. 